# Install dependency

In [ ]:
!pip install simpletransformers 

# Import ClassificationModel

- simpletransformers provides abstractions for `torch` and `transformers` (Hugging Face) implementations 

In [2]:

from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

# Define function to load and (optional) split fnc data into training and val

In [33]:
import os
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

FNC1_DATA_PATH = '/content/drive/MyDrive/fnc-1'

STANCE_2_ID = {'agree': 0, 'disagree': 1, 'discuss': 2, 'unrelated': 3}

SENTENCE_PAIR_COLS = ['text_a', 'text_b', 'labels']
def combine_headline_body_and_split_train_val(body_path, headline_path, split=True, body_dict={}):
    body_csv_df = pd.read_csv(body_path)
    df = body_csv_df.reset_index()
    for index, row in body_csv_df.iterrows():
        body_dict[row["Body ID"]] = row["articleBody"]

    headlines, bodies, labels = [], [], []
    headline_csv_df = pd.read_csv(headline_path)
    df = headline_csv_df.reset_index()
    for index, row in headline_csv_df.iterrows():
        headlines.append(row["Headline"])
        bodies.append(body_dict[row["Body ID"]])
        labels.append(STANCE_2_ID[row["Stance"]])

    combined_df = pd.DataFrame(list(zip(headlines, bodies, labels)), columns=SENTENCE_PAIR_COLS)
    if not split:
      labels_df = pd.Series(combined_df['labels']).to_numpy()
      return combined_df, labels_df

    train_df, val_df = train_test_split(combined_df)
    return train_df, val_df, pd.Series(val_df['labels']).to_numpy()


# Define Evaluate Model Function to calculate F1 scores and accurary

In [32]:
from sklearn.metrics import f1_score

LABELS = [0, 1, 2, 3]
RELATED = [0, 1, 2]
CONFUSION_MATRIX = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]


def calc_f1(real_labels, predicted_labels):
    f1_macro = f1_score(real_labels, predicted_labels, average='macro')
    f1_classwise = f1_score(real_labels, predicted_labels, average=None, labels=[0, 1, 2, 3])
    return f1_macro, f1_classwise


def calculate_accuracy(predicted_labels, real_labels):
    score = 0.0
    cm = CONFUSION_MATRIX
    for i, (g, t) in enumerate(zip(predicted_labels, real_labels)):
      cm[g][t] += 1
    
    hit, total = 0, 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
    return (hit / total)*100

def evaluate_model(model, test_df):
    _, outputs, _ = model.eval_model(test_df)
    predictions = np.argmax(outputs, axis=1)
    print(calc_f1(predictions, test_labels))
    print(calculate_accuracy(predictions, test_labels))

# Load Training and Val Data and Labels

In [25]:
train_df, val_df, labels_val = combine_headline_body_and_split_train_val(
    os.path.join(FNC1_DATA_PATH, 'train_bodies.csv'),
    os.path.join(FNC1_DATA_PATH, 'train_stances.csv'),
)

# Load Competition Test Data and Labels

In [23]:
test_df, test_labels = combine_headline_body_and_split_train_val(
    os.path.join(FNC1_DATA_PATH, 'competition_test_bodies.csv'),
    os.path.join(FNC1_DATA_PATH, 'competition_test_stances.csv'),
    split=False
)

# Train and Tune Model with BERT

In [ ]:
bert_model = ClassificationModel(
    'bert', 
    'bert-base', 
    use_cuda=True,
    num_labels=4, 
    args={
      'fp16': True, 
       # Tune hyperparameter 3e-4, 1e-4, 5e-5, 3e-5
      'learning_rate':3e-5,
      'num_train_epochs': 4,
      'reprocess_input_data': True,
      'overwrite_output_dir': True,
      'process_count': 10,
      'train_batch_size': 8,
      'eval_batch_size': 8,
      'max_seq_length': 512
    # 'output_dir': ''
})

# TRAIN
bert_model.train_model(train_df)
evaluate_model(bert_model, test_labels)

# TUNE
bert_model.train_model(val_df)
evaluate_model(bert_model, labels_val)

# Train and Tune Model with RoBERTa

> Indented block



In [ ]:
roberta_model = ClassificationModel(
    'roberta', 
    'roberta-base', 
    use_cuda=True,
    num_labels=4, 
    args={
      'fp16': True, 
      # Tune hyperparameter 3e-4, 1e-4, 5e-5, 3e-5
      'learning_rate':5e-5,
      'num_train_epochs': 4,
      'reprocess_input_data': True,
      'overwrite_output_dir': True,
      'process_count': 10,
      'train_batch_size': 8,
      'eval_batch_size': 8,
      'max_seq_length': 512,
    # 'output_dir': ''
})

# TRAIN
roberta_model.train_model(train_df)
evaluate_model(roberta_model, test_labels)

# TUNE
roberta_model.train_model(val_df)
evaluate_model(roberta_model, labels_val)

## Generate Competition Submission Prediction


In [29]:
import os
import csv
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

FNC1_DATA_PATH = '/content/drive/MyDrive/fnc-1'

body_dict = {}
body_csv_df = pd.read_csv(os.path.join(FNC1_DATA_PATH, 'competition_test_bodies.csv'))
df = body_csv_df.reset_index()
for index, row in body_csv_df.iterrows():
    body_dict[row["Body ID"]] = row["articleBody"]

headlines, bodies, combined_headline_bodies = [], [], []
headline_csv_df = pd.read_csv(os.path.join(FNC1_DATA_PATH, 'competition_test_stances_unlabeled.csv'))
df = headline_csv_df.reset_index()
for index, row in headline_csv_df.iterrows():
    headlines.append(row["Headline"])
    bodies.append(row["Body ID"])
    combined_headline_bodies.append(row["Headline"], body_dict[row["Body ID"]])

predictions, raw_outputs = roberta_model.predict(combined_headline_bodies)

# Store and format submission csv

In [ ]:
df = pd.DataFrame(list(zip(headlines, bodies, predictions)), columns=['Headline', 'Body ID', 'Stance'])
df['Stance'] = df['Stance'].replace({0: 'agree', 1: 'disagree', 2: 'discuss', 3: 'unrelated'})
df.to_csv('answer.csv', index=False, encoding='utf-8') # From pandas library